In [1]:
import egttools as egt
import numpy as np
import math
import matplotlib.pyplot as plt
import scipy as sp

import run
from run import CRDWithExecutor, Cooperator, Defector, Executor

Z = 100         # Population size
N  = 4           # Group size
b  = 1.           # Endowment (individual's money/funds/...)
c  = 0.1         # Amount of money individuals contribute
Mc = 0.3         # Minimum collective contribution
M  = 3.           # OR Minimum number of cooperators
r  = 0.2         # If minimum is not met: All group participants lose their endowment with probability r, else: individuals retain their endowments
pi_t = 0.03
pi_e = 0.3
n_e = 0.25
alpha = 1.
mu    = 1/Z
beta = 5.

game = CRDWithExecutor(
    strategies=[Defector(c, b), Executor(c, b, pi_t, pi_e, alpha), Cooperator(c, b)],
    initial_endowment=b,
    population_size=Z,
    group_size=N,
    cost=c,
    risk=r,
    alpha=alpha,
    cooperation_threshold=M,
    enhancement_factor=1,
    pi_t=pi_t,
    pi_e=pi_e,
    n_e=n_e,
    mu=mu)

In [2]:
# Make population and group
current_population = np.array([33, 34, 33], dtype=int)
pop_id = egt.calculate_state(100, current_population)

dist = sp.stats.multivariate_hypergeom(current_population, game.N)
group = dist.rvs()[0]
group_id = egt.calculate_state(game.N, group,)

print(group)

[1 1 2]


In [3]:
# Test fitness difference
game.set_population_state(pop_id)
payoffs = game.calculate_payoffs()
evolver = egt.analytical.StochDynamics(
        3, 
        np.array(payoffs), 
        pop_size=game.Z, 
        group_size=game.N, 
        mu=game.mu)

evolver.full_fitness_difference_group(0, 2, population_state=current_population)

-0.20397834860279696

In [4]:
# Generating a random pair of 2 strategies from population
sample = sp.stats.multivariate_hypergeom.rvs(m=current_population, n=2)
print(sample)
idvs = [0 for _ in range(sample[0])]+[1 for _ in range(sample[1])]+[2 for _ in range(sample[2])]
np.random.shuffle(idvs)
print(idvs)

[1 1 0]
[1, 0]


In [6]:
# Calculating group achievement from stationary distribution estimate
import pickle
with open("fig1/alpha=0.pickle", 'rb') as f:
        sd = pickle.load(f)

group_achievement = sum([
        sd[i]*game.aG(i) for i in range(len(sd))
    ])
print("Alpha = 0:",group_achievement)

with open("fig1/alpha=1.pickle", 'rb') as f:
        sd = pickle.load(f)

group_achievement = sum([
        sd[i]*game.aG(i) for i in range(len(sd))
    ])
print("Alpha = 1:",group_achievement)

Alpha = 0: 0.6432707821631969
Alpha = 1: 0.9902054389878037


In [9]:
import pickle
with open("0.pickle", 'rb') as f:
        sd = pickle.load(f)

group_achievement = sum([
        sd[1][i]*game.aG(i) for i in range(len(sd[1]))
    ])
print("Alpha = 0:",group_achievement)

with open("1.pickle", 'rb') as f:
        sd = pickle.load(f)

group_achievement = sum([
        sd[1][i]*game.aG(i) for i in range(len(sd[1]))
    ])
print("Alpha = 1:",group_achievement)

Alpha = 0: 0.9253718835267142
Alpha = 1: 0.9993971055806767


In [10]:
from simulate import estimate_stationary_distribution

Z = 100         # Population size
N  = 4           # Group size
b  = 1.           # Endowment (individual's money/funds/...)
c  = 0.1         # Amount of money individuals contribute
Mc = 0.3         # Minimum collective contribution
M  = 3.           # OR Minimum number of cooperators
r  = 0.2         # If minimum is not met: All group participants lose their endowment with probability r, else: individuals retain their endowments
pi_t = 0.03
pi_e = 0.3
n_e = 0.25
alpha = 1.
mu    = 1/Z
beta = 5.

transitory = 10**2      # num of steps before we start counting
nb_generations = 10**4  # num of steps where we do count
nb_runs = 10            # num of different runs we average over

for alpha in [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]:
        game = CRDWithExecutor(
            strategies=[Defector(c, b), Executor(c, b, pi_t, pi_e, alpha), Cooperator(c, b)],
            initial_endowment=b,
            population_size=Z,
            group_size=N,
            cost=c,
            risk=r,
            alpha=alpha,
            cooperation_threshold=M,
            enhancement_factor=1,
            pi_t=pi_t,
            pi_e=pi_e,
            n_e=n_e,
            mu=mu)
        beta = 5
        payoffs = game.calculate_payoffs()

        evolver = egt.analytical.StochDynamics(
            3, 
            np.array(payoffs), 
            pop_size=game.Z, 
            group_size=game.N, 
            mu=game.mu)
     
        #incredibly slow + bugged in egttools 1.11
        #stationary_distribution = evolver.calculate_stationary_distribution(beta=beta)
        stationary_distribution = estimate_stationary_distribution(
            game=game,
            nb_runs=nb_runs,
            transitory=transitory,
            nb_generations=nb_generations,
            beta=beta,
            mu=mu,
            Z=Z,
        )
        group_achievement = sum([
            stationary_distribution[i]*game.aG(i) for i in range(len(stationary_distribution))
        ])

        print(f'r={r}_alpha={alpha} => ', group_achievement)

        with open(f'fig2A_r={r}_alpha={alpha}.pickle', 'wb') as f:
            pickle.dump([payoffs, stationary_distribution, group_achievement], f)


		--- Run 0 ---
Initial state:	 [ 7 68 25]
After transitory period:	 [ 6 57 37]
Final state:	 [ 7  4 89]
		--- Run 1 ---
Initial state:	 [17 79  4]
After transitory period:	 [13 83  4]
Final state:	 [11 10 79]
		--- Run 2 ---
Initial state:	 [95  4  1]
After transitory period:	 [96  2  2]
Final state:	 [97  3  0]
		--- Run 3 ---
Initial state:	 [29 54 17]
After transitory period:	 [21 54 25]
Final state:	 [ 4  2 94]
		--- Run 4 ---
Initial state:	 [70 21  9]
After transitory period:	 [72 15 13]
Final state:	 [30  2 68]
		--- Run 5 ---
Initial state:	 [ 8 19 73]
After transitory period:	 [10 22 68]
Final state:	 [96  0  4]
		--- Run 6 ---
Initial state:	 [31 42 27]
After transitory period:	 [28 42 30]
Final state:	 [69  0 31]
		--- Run 7 ---
Initial state:	 [13 16 71]
After transitory period:	 [11  8 81]
Final state:	 [ 4 29 67]
		--- Run 8 ---
Initial state:	 [13 30 57]
After transitory period:	 [ 9 35 56]
Final state:	 [13  3 84]
		--- Run 9 ---
Initial state:	 [82  9  9]
After transi